# Skywalker algorithm against the lcls mirror systems

In [1]:
SIMULATION = True
ALIGNMENT = 'HOMS'
#ALIGNMENT = 'MEC'
#ALIGNMENT = 'MFX'
#ALIGNMENT = 'XCS'

alignment_goals = dict(HOMS=[210, 270],
                       MEC=[270],
                       MFX=[270],
                       XCS=[270, 270])
first_steps = 10 # Naive first step for the parameters search
tolerances = 2.0 # Tolerance of each target in Pixels
average = 1 # Number of shots to average over
log_level = "INFO"

# PARAMETERS FOR SIMULATION ONLY
centroid_noise = 5.0 # Noise level of centroid measurements
infinite_yag = False # Assume all yags are infinitely large

In [2]:
############
# Standard #
############
import os
import sys
import logging
import argparse

###############
# Third Party #
###############
from bluesky import RunEngine
from bluesky.tests.utils import MsgCollector
from bluesky.plans import run_wrapper
from pcdsdevices.sim import pim, source, mirror

In [3]:
"""
Parameters
----------
"""
#Configure logger
log_level = getattr(logging, log_level, None)

#Report invalid logging level
if not isinstance(log_level, int):
    raise ValueError("Invalid log level : {}".format(log_level))

#Create basic configuration
logging.basicConfig(level=log_level,
                format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')

from pswalker.examples  import patch_pims
from pswalker.skywalker import skywalker
from pswalker.plans import walk_to_pixel

#Instantiate simulation
if not SIMULATION:
    print("*"*80)
    print("* WARNING: Running on real devices in Production...")
    print("*"*80)
    input("Press any key to continue...")
    system = homs_system()
    m1h = system['m1h']
    m1h2 = system['m1h2']
    m2h = system['m2h']
    m2h2 = system['m2h2']
    xrtm2 = system['xrtm2']
    xrtm22 = system['xrtm22']
    hx2 = system['hx2']
    dg3 = system['dg3']
    mfxdg1 = system['mfxdg1']
    mecy1 = system['mecy1']
    m1 = m1h
    m2 = m2h
    y1 = hx2
    y2 = dg3
else:
    s = source.Undulator('test_undulator')
    m1 = mirror.OffsetMirror('test_m1h', z=90.510, alpha=0.0014)
    m2 = mirror.OffsetMirror('test_m2h', x=0.0317324, z=101.843, alpha=0.0014)
    xrtm2 = mirror.OffsetMirror('test_xrtm2', x=0.0317324, z=200, alpha=0.0014)
    y1 = pim.PIM('test_p3h', x=0.0317324, z=103.660,
             zero_outside_yag= not infinite_yag)
    y2 = pim.PIM('test_dg3', x=0.0317324, z=375.000,
             zero_outside_yag= not infinite_yag)
    mecy1 = pim.PIM('test_mecy1', x=0.0317324, z=350,
             zero_outside_yag= not infinite_yag)
    mfxdg1 = mecy1
    patch_pims([y1, y2], mirrors=[m1, m2], source=s)
    patch_pims([mecy1], mirrors=[xrtm2], source=s)

    #Add noise
    y1.centroid_noise = centroid_noise
    y2.centroid_noise = centroid_noise
    mecy1.centroid_noise = centroid_noise

alignment_yags = dict(HOMS=[y1, y2],
                      MEC=[mecy1],
                      MFX=[mfxdg1],
                      XCS=['pbty', 'xcssb1y'])
alignment_mots = dict(HOMS=[m1, m2],
                      MEC=[xrtm2],
                      MFX=[xrtm2],
                      XCS=['xrtm1', 'xrtm3'])

2017-07-26 12:59:22,071 - INFO - pswalker.examples - Patching 2 pim(s)
2017-07-26 12:59:22,072 - INFO - pswalker.examples - Patching 1 pim(s)


In [4]:
#Create Skywalker plan
yags = alignment_yags[ALIGNMENT]
mots = alignment_mots[ALIGNMENT]
goals = alignment_goals[ALIGNMENT]
det_rbv = 'detector_stats2_centroid_x'
mot_rbv = 'pitch'
plan = skywalker(yags, mots, det_rbv, mot_rbv,
        goals, first_steps=first_steps, tolerances=tolerances,
        averages=average, timeout=10)  

In [5]:
#Create RunEngine
RE = RunEngine({})
if SIMULATION:
    collector = MsgCollector()
    RE.msg_hook = collector

In [6]:
RE(run_wrapper(plan))

2017-07-26 12:59:29,106 - INFO - pswalker.plans - Averaged data yielded TST:test_mecy1_detector_stats2_centroid_x is at 2993.1685147691433
2017-07-26 12:59:29,108 - INFO - pswalker.callbacks - Unable to yield estimate from model Model(linear)
2017-07-26 12:59:29,109 - INFO - pswalker.plans - Walking motor, error after step #0 -> -2723.1685147691433
2017-07-26 12:59:29,110 - INFO - pswalker.plans - No model yielded accurate prediction, using naive plan
2017-07-26 12:59:29,244 - INFO - pswalker.plans - Averaged data yielded TST:test_mecy1_detector_stats2_centroid_x is at 3244.887042483765
2017-07-26 12:59:29,246 - INFO - pswalker.plans - Walking motor, error after step #1 -> -2974.887042483765
2017-07-26 12:59:29,247 - INFO - pswalker.plans - Using model Model(linear) to determine next step.
2017-07-26 12:59:29,249 - INFO - pswalker.plans - Adjusting motor MIRR:TST:test_xrtm2 to position -108.18167811876502
2017-07-26 12:59:29,361 - INFO - pswalker.plans - Averaged data yielded TST:test_

['cede5e6d-a5a3-4de7-9c5a-8d03464ef83c']

In [7]:
if SIMULATION:
    #Analyze Performance
    print(len(RE.msg_hook.msgs))

47
